# Linear SVM Classification

The decision boundary of an SVM classifier not only separates the two classes but also stays as far away as possible from the closest training instances. It is like fitting the widest possible street between the classes. This is called *large margin classification*. 

Adding more training instances off the street do not affect the decision boundary at all, it is fully determined by the instances located on the edge of the street. These instances are called the *support vectors*.

SVMs are sensitive to feature scales, so the features must be scaled using the **StandardScaler** before the model is fit on the training data.

SVMs are good for classification on small to medium sized datasets.

### Soft Margin Classification

If we strictly impose that all instances must be off the street this is called *hard margin classification*. There are two main issues with hard margin classification.

1. It only works if the data is linearly separable.
2. It is sensitive to outliers and will not generalize well.

To avoid these issues, we use a more flexible model. The objective is to find a good balance between keeping the street as large as possible and limiting the *margin violations*. This is called *soft margin classification*. 

When creating an SVM model using Scikit-Learn, we can specify the hyperparameter **C**. If we set it to a lowe value, we end up with a model that is more soft margin than hard margin and vice versa. If an SVM model is overfitting, we can try regularizing it by reducing C.

The following code loads the iris dataset, scales the features, and then trains a linear SVM model to detect *Iris virginica* flowers.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [3]:
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris virginica

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge"))
])

svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [5]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

Unlike Logistic Regression, SVM classifiers do not output probabilities for each class.

Instead of using the **LinearSVC** class, we could use the SVC class with a linear kernel. When creating the SVC model, we would write **SVC(kernel="linear", C=1)**. Or we could use the **SGDClassifier** class with **SGDClassifier(loss="hinge", alpha=1/(m * C))**. This applies regular Stochastic Gradient Descent to train a linear SVM classifier. It does not converge as fast as the **LinearSVC** class, but it can be used for out-of-core training.

It is important to set the loss hyperparameter to "hinge", as it is not the default value. For better performance, we should also set the **dual** hyperparameter to **False**, unless there are more features than training instances.

# Nonlinear SVM Classification

One approach to handling nonlinear datasets is to add polynomial features which may result in a linearly separable daatset.

The code below implements the above idea on the moons dataset.

In [6]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
])

polynomial_svm_clf.fit(X, y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

### Polynomial Kernel

Adding polynomial features is a good method but fails for data that is too complex since we cannot create a very degree polynomial because it may lead to feature explosion.

SVMs have a workaround this with the *kernel trick*. It gets the exact same result as polynomial features except it doesn't actually add any features and only does the math using these extra features in the backend. 

In [7]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])

poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

The hyperparameter **coef0** controls how much the model is influenced by high-degree polynomials versus low-degree polynomials

### Similarity Features

Another technique to tackle nonlinear problems is to add features computed using a *similarity function*, which measures how much an instance resembles a particular *landmark*. For example, let's assume a 1D dataset and add two landmarks to it at $x_{1} = -2$ and $x_{1} = 1$. Defining the similarity function to be the Gaussian *Radial Basis Function* (RBF) with $\gamma = 0.3$, the RBF equation is:

$\phi_{\gamma} (x, l) = exp(- \gamma || x - l ||^{2})$

This is a bell-shaped function varying from 0 to 1. Now we are ready to compute the new features. Assume there is an instance $x_{1} = -1$, it is located at a distance of 1 from the first landmark and 2 from the second landmark. Therefore, its new features are $x_{2} = exp(-0.3 * 1^{2}) = 0.74$ and $x_{3} = exp(-0.3 * 2^{2}) = 0.30$.

As for how to select landmarks, the simplest approach would be to create a landmark at the location of each and every instance in the dataset. Doing that creates many dimensions and increases the chance that the transformed training set is linearly separable. The downside is that a training set with *m* instances and *n* features gets converted to a training set with *m* instances and *m* features. If the training set is very large, we end up with an equally large number of features.

### Gaussian RBF Kernel

Just like polynomial features, the similarity features method also causes a feature explosion. Once again, SVM's kernel trick comes clutch and doesn't cause a feature explosion. 

In [8]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])

rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

This classifier has the hyperparemters **gamma** and **C**. Increasing **gamma** makes the bell-shaped curve narrower. As a result, each instance's range of influence is smaller, the decision boundary ends up being more irregular, wiggling around individual instances. Conversely, a small **gamma** makes the bell-shaped curve wider, instances have a large range of influence and the decision boundary ends up smoother. So $\gamma$ acts like a regularization hyperparaameter. If the model is overfitting, it should be reduced; if it is unerfitting, it should be increased.

The rule of thumb is to test out a LinearSVM or linear kernel SVM first, especially when the training set is large. If not, a Gaussian RBF kernel is the next best option.

*Comparison of Scikit-Learn classes for SVM Classification*

| Class    | Time Complexity | Out-of-core support | Scaling required | Kernel trick |
| ---      |     :---:       | :---:               | :---:            | ---:         |
| Linear SVC | $O(m * n)$ | No | Yes | No |
| SGDClassifier | $O(m * n)$ | Yes | Yes | No |
| SVC | $O(m^{2} * n)$ to $O(m^{3} * n)$ | No | Yes | Yes|

# SVM Regression

The SVM algorithm supports linear and nonlinear regression too. To use SVMs for regression instead of classification, we must reverse the objective. Instead of trying to fit the largest possible street between the two classes while limiting margin violations, SVM Regression tries to fit as many instances as possible *on* the street while limiting margin violations. The width of the street is controlled by the hyperparameter $\epsilon$. Adding more training instances within the margin does not affect the model's predictions thus the model is said to be *$\epsilon$-insensitive*

In [10]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5)

In [11]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)

SVR(C=100, degree=2, kernel='poly')

SVMs can also be used for outlier detection.

# Under The Hood

### Decision Functions and Predictions

The linear SVM classifer predicts the class of a new instance by simply computing the decision function $w^{T}x + b$. If the result is positive the predicted class $\hat{y}$ is the positive class (1), else the negative class (0).

The decision function corresponding to a data with two features is a 2D plane. The decision boundary is the set of points where the decision function is equal to 0, it is the intersection of two planes which is a straight line. The two support vectors are where the decision function is equal to -1 and 1. Training a linear SVM classifier means finding the values of **w** and **b** that make this margin as wide as possible while avoiding margin violations (hard margin) or limiting them (soft margin)

### Training Objective

The slope of the decision function is equal to the norm of the weight vector, **$||w||$.** If we divide this slope by 2, the points where the decision function is equal to -1 and 1 are going to be twice as far away from the decision boundary. Dividing the slope by 2 will multiply the margin by 2. The smaller the weight vector **w**, the larger the margin.

So we want to minimize $||w||$ to get a large margin. If we also want to avoid any margin violations (hard margin), then we need the decision function to be greater than 1 for all positive instances and less than -1 for all negative instances. If we define $t^{(i)} = -1$ for negative instances and $t^{(i)} = 1$ for all positive instances then we can express this constraint as $t^{(i)} (w^{T} x^{(i)} + b) \geq 1$ for all instances.

We can therefore express the hard margin linear SVM classifier objective as the constrained optimization problem:

$minimize_{w, b} \frac{1}{2} w^{T} w$

subject to $t^{(i)} (w^{T} x^{(i)} + b) \geq 1$ for all instances

We are minimizing $\frac{1}{2} w^{T} w$, which is equal to $\frac{1}{2} ||w||^{2}$ rather than minimizing $||w||$ because it has a nice, simple derivative, while $||w||$ is not differentiable at $w=0$. Optimization algorithms work much better on differentiable functions.

To get the soft margin objective, we need to introduce a *slack variable* $\zeta^{i} \geq 0$, for each instance, it measures how much an instance is allowed to violate the margin. We now have two conflicting objectives: make the slack variables as small as possible to reduce the margin violations and make $\frac{1}{2} w^{T} w$ as small as possible to increase the margin. The **C** hyperparameter allows us to define the trade-off between these two objectives. This gives us the constrained optimization problem:

$minimize_{w, b, \zeta} \frac{1}{2} w^{T} w + C \sum_{i=1}^{m} \zeta^{(i)}$

subject to $t^{(i)} (w^{T} x^{(i)} + b) \geq 1 - \zeta^{(i)} and \zeta^{(i)} \geq 0$ for all instances.

### Quadratic Programming

The hard margin and soft margin problems are both convex quadratic optimization problems with linear constraints. Such problems are called *Quadratic Programming* (QP) problems. Many solvers are available to solve QP problems.

### The Dual Problem

Given a constrained optimization problem, known as the *primal problem*, it is possible to express a different but closely related problem, called its *dual problem*. The solution to the dual problem typically gives a lower bound to the solution of the primal problem, but unders some conditions it can have the same solution as the primal problem. The SVM problem happens to meet these conditions (the objective function is convex, and the inequality constraints are continuously differentiable and convex functions), so we can choose to solve the primal problem or the dual problem; both will have the same solution. *The dual form of the linear SVM objective*:

$minimize_{\alpha} \frac{1}{2} \sum_{i=1}^{m} \sum_{j=1}^{m} \alpha^{(i)} \alpha^{(j)} t^{(i)} t^{(j)} x^{(i)T} x^{(j)} - \sum_{i=1}^{m} \alpha^{(i)}$

subject to $\alpha^{(i)} \geq 0$ for all instances i equals 1 to m.

Once we find the vector $\hat{\alpha}$ that minimizes this equation using a QP solver, we can use the below equations to compute $\hat{w}$ and $\hat{b}$ that minimize the primal problem. *From the dual solution to the primal solution*:

$\hat{w} = \sum_{i=1}^{m} \hat{\alpha}^{(i)} t^{(i)} x^{(i)}$

$\hat{b} = \frac{1}{n_{s}} \sum_{i=1}^{m} (t^{(i)} - \hat{w}^{T} x^{(i)})$

The dual problem is faster to solve than the primal one when the number of training instances is smaller than the number of features. More importantly, the dual problem makes the kernel trick possible, while the primal does not.

### Kernelized SVMs

Suppose we want to apply a second-degree transformation to a two-dimensional training set (such as the moons training set), then train a linear SVM classifier on the transformed training set. The second-degree polynomial mapping $\phi$ is given by:

$\phi(x) = [\phi(x_{1}), \phi(x_{2})] = [x_{1}^{2}, \sqrt{2} x_{1} x_{2}, x_{2}^{2}]$

The transformed vector is 3D instead of 2D. Let's see what happens to a couple of 2D vectors, **a** and **b**, if we apply a second degree transformation and then compute the dot product of the tranformed vectors:

$\phi(a)^{T} \phi(b) = (a_{1} b_{1} + a_{2} b_{2})^{2} = (a^{T}b)^{2}$

The dot product of the tranformed vectors is equal to the square of the dot product of the original vectors:

$\phi(a)^{T} \phi(b) = (a^{T}b)^{2}$

The key insight is that if we apply the transformation $\phi$ to all training instances then the dual problem will contain the dot product $\phi(x^{(i)})^{T} \phi(x^{(j)})$. But if $\phi$ is the second-degree polynomial transformation then we can replace this dot product of transformed vectors simply by $(x^{(i)T} x^{(j)})^{2}$. So we don't need to transform the training instances at all; just replace the dot product by its square. 

The function $K(a, b) = (a^{T}b)^{2}$ is a second-degree polynomial kernel. In ML, a *kernel* is a function capable of computing the dot product based only on the original vectors, without having to know about the transformation $\phi$. *Common kernels*:

Linear: $K(a, b) = a^{T}b$

Polynomial: $K(a, b) = (\gamma a^{T}b + r)^{d}$

Gaussian RBF: $K(a, b) = exp(- \gamma ||a - b||^{2})$

Sigmoid: $K(a, b) = tanh(\gamma a^{T}b + r)$

There is still one loose end. The equations above show how to go from the dual solution to the primal solution in the case of a linear SVM classifier. But if we apply the kernel trick, we end up with equations that must include $\phi(x^{(i)})$. In fact, $\hat{w}$ must have the same number of dimensions as $\phi(x^{(i)})$, which may be huge or even infinite, so we can't compute it. The work around is that we can plug the formula for $\hat{w}$ into the decision function for a new instance $x^{(n)}$, we can get an equation with only dot products between input vectors. *Making predictions with a kernelized SVM*:

$h_{\hat{w}, \hat{b}} (\phi(x^{(n)})) + \hat{b} = \sum_{i=1}^{m} \hat(\alpha)^{(i)} t^{(i)} K(x^{(i)}, x^{(n)}) + \hat{b}$.

We can use the same trick to compute $\hat{b}$

### Online Loss

For SVM classifiers, one method for implementing an online SVM classifier is to use Gradient Descent (**SGDClassifier**) to minimize the cost function:

$J(w, b) = \frac{1}{2} w^{T} w + C \sum_{i=1}^{m} max(0, 1 - t^{(i)} (w^{T} x^{(i)} + b))$

which is derived from the primal problem. Unfortunately, Gradient Descent converges much more slowly than the methods based on QP.

The first sum in the cost function will push the model to have a small weight vector **w**, leading to a larger margin. The second sum computes the total of all margin violations. An instance's margin violation is equal to 0 if it is located off the street on the correct side, or else it is proportional to the distance of the correct side of the street. Minimizing this term ensures that the model makes the margin violations as small and as few as possible.

### Hinge Loss

The function $max(0, 1-t)$ is called the *hinge loss* function. It is equal to 0 when $t \geq 1$. Its derivative (slope) is equal to -1 if $t \leq 1$ and 0 if $t \geq 1$. It is not differentiable at $t=1$, but just like for Lasso Regression, we can still use Gradient Descent using any *subderivative* at $t=1$.